In [3]:
import configparser
import psycopg2
from time import time
from sql_queries import copy_table_queries, insert_table_queries
from create_tables import create_tables, drop_tables

In [4]:
def load_staging_tables(cur, conn):
    for query in copy_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()


def insert_tables(cur, conn):
    for query in insert_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()

In [5]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()
    
print("Droping tables")
drop_tables(cur, conn)
print("Creating tables")
create_tables(cur, conn)

print("Loading staging tables")
load_staging_tables(cur, conn)

print("Loading final tables")
insert_tables(cur, conn)

conn.close()

 COPY staging_events FROM 's3://udacity-dend/log_data'
        iam_role 'arn:aws:iam::909496952025:role/dwhRole'
        JSON 'auto ignorecase' ACCEPTINVCHARS
        
 COPY staging_songs FROM 's3://udacity-dend/song_data'
        iam_role 'arn:aws:iam::909496952025:role/dwhRole'
        JSON 'auto ignorecase' ACCEPTINVCHARS
        
DROP TABLE IF EXISTS users
DROP TABLE IF EXISTS songs

        INSERT INTO artists (artist_id, name, location, latitude, longitude)
        SELECT
            DISTINCT (artist_id) as artist_id,
            artist_name as name,
            artist_location as location,
            artist_latitude as latitude,
            artist_longitude as longitude 
        FROM staging_songs
        WHERE artist_id IS NOT NULL

